In [2]:
import pandas as pd

In [6]:
#read weather csv as dataframe
weather_data = pd.read_csv('../data/csv/final_weather_data.csv')

#check for missing values
non_missing_weather = pd.notnull(weather_data)

#filter rows where data is not missing
weather_data = weather_data[non_missing_weather]

#convert Time column do datetime format
weather_data['Time'] = weather_data['Time'].str.replace('T', ' ')
weather_data['Time'] = pd.to_datetime(weather_data['Time'])

#create month-year column from Time column
#weather_data['month-year'] = weather_data['Time'].dt.strftime('%Y-%m')

#calculate median of each weather variable by month
monthly_medians = (weather_data.groupby(weather_data['Time'].dt.strftime('%B-%Y'))[['Temperature (C)', 'Relative Humidity (%)', 'Precipitation (mm)', 'Surface Pressure (hPa)']].median()).reset_index()

#re-convert Time column to datetime to sort chronologically
monthly_medians['Time'] = pd.to_datetime(monthly_medians['Time'])

#save monthly medians into csv
monthly_medians = monthly_medians.sort_values("Time").reset_index(drop=True)
monthly_medians.to_csv('../data/csv/monthly_medians.csv', float_format='%.2f', index=False,header=True)

#calculate median of each weather variable by year
yearly_medians = (weather_data.groupby(weather_data['Time'].dt.strftime('%Y'))[['Temperature (C)', 'Relative Humidity (%)', 'Precipitation (mm)', 'Surface Pressure (hPa)']].median()).reset_index()

#save yearly medians to csv
yearly_medians.to_csv('../data/csv/yearly_medians.csv', index=False,header=True)

#change Time column name to Year
yearly_medians = yearly_medians.rename(columns = {"Time": "Year"})
yearly_medians.Year = yearly_medians.Year.astype(int)

#read csv harvest data
harvest_data = pd.read_csv("../data/csv/br_final.csv")

#filter harvest data for Minas Gerais in 2022 and 2023 and keep columns (million_60kgs_bag, nonbear_mill_trees, bear_mill_trees, avg_unemp_perc)
mg_harvest = harvest_data[(harvest_data['subdivision'] == 'Minas Gerais') & (harvest_data['year'].isin([2022, 2023]))][['year','million_60kgs_bag', 'nonbear_mill_trees', 'bear_mill_trees', 'avg_unemp_perc']]
mg_harvest = mg_harvest.rename(columns = {'year': 'Year'}).reset_index(drop = True)

#merge harvest data and yearly weather data
yearly_data = pd.merge(yearly_medians, mg_harvest, how="left", on="Year")

#save yearly data to csv
yearly_data.to_csv('../data/csv/yearly_data_weather_harvest.csv', index=False, header=True)

display(yearly_data)




/var/folders/pg/vf32t04x6td1g_z1bnn3rg0c0000gn/T/ipykernel_77919/1647748506.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  monthly_medians['Time'] = pd.to_datetime(monthly_medians['Time'])


,Year,Temperature (C),Relative Humidity (%),Precipitation (mm),Surface Pressure (hPa),million_60kgs_bag,nonbear_mill_trees,bear_mill_trees,avg_unemp_perc
0,2022,18.6,78.0,0.0,911.4,24.5,1500.0,6010.0,7.15
1,2023,20.7,78.0,0.0,913.4,28.5,1510.0,6100.0,NaN
